## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sfacts as sf

In [ ]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy as sp
import pyro
import pyro.distributions as dist
import torch
from functools import partial
from tqdm import tqdm
import xarray as xr
import warnings
from torch.jit import TracerWarning

mpl.rcParams['figure.dpi']= 120

In [ ]:
warnings.filterwarnings(
    "ignore",
    message="torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.",
    category=TracerWarning,
#     module="trace_elbo",  # FIXME: What is the correct regex for module?
#     lineno=5,
)

## Experiments

### Experiment 0: Average and variation in accuracy

In [ ]:
results = []
for seed in range(10):
    for _ in range(1):  # Dummy loop to match other experiments.
        generr, comperr, mcomperr, entropy, runtime, sim, fit = sf.workflow.simulate_fit_and_evaluate(
            s_sim=30,
            n_sim=100,
            g_sim=500,
            n_fit=100,
            g_fit=500,
            sim_kwargs=dict(
                data=dict(
                    alpha_hyper_mean=100.,
#                     rho=np.logspace(0, 1, num=20) / np.logspace(0, 1, num=20).sum(),
                ),
                gamma_hyper=0.01,
                delta_hyper_temp=0.01,
                delta_hyper_p=0.7,
                pi_hyper=0.5,
                rho_hyper=5.,
                mu_hyper_mean=1.,
                mu_hyper_scale=0.5,
#                 m_hyper_r=15.,
                alpha_hyper_scale=0.5,
                epsilon_hyper_alpha=1.5,
                epsilon_hyper_beta=1.5/0.01,
                device='cpu'
            ),
            preclust=True,
            preclust_kwargs=dict(
                thresh=0.1,
                additional_strains_factor=0.1,
                progress=False,
            ),
            fit_kwargs=dict(
#                 s=30,
                gamma_hyper=0.01,
                pi_hyper=0.5,
                rho_hyper=0.05,
                mu_hyper_mean=5,
                mu_hyper_scale=5.,
#                 m_hyper_r=5.,
                delta_hyper_temp=0.1,
                delta_hyper_p=0.9,
                alpha_hyper_hyper_mean=100.,
                alpha_hyper_hyper_scale=10.,
                alpha_hyper_scale=0.5,
                epsilon_hyper_alpha=1.5,
                epsilon_hyper_beta=1.5 / 0.01,
                device='cpu',
                lagA=10,
                lagB=50,
                opt=pyro.optim.Adamax({"lr": 1e-0}),
                progress=True
            ),
            postclust=True,
            postclust_kwargs=dict(
                thresh=0.1,
            ),
            seed_sim=seed,
            seed_fit=seed,
            quiet=True,
        )
        results.append((seed, generr, comperr, mcomperr, entropy, runtime))
        print(seed, generr, comperr, mcomperr, entropy, runtime, sep='\t')
         
results0 = pd.DataFrame(results, columns=['seed', 'generr', 'comperr', 'mcomperr', 'entropy', 'runtime'])

In [ ]:
fig, axs = plt.subplots(3, 2)

for stat, ax in zip(['generr', 'comperr', 'mcomperr', 'entropy', 'runtime'], axs.flatten()):
    ax.hist(results0[stat])
    ax.set_title(stat)
fig.tight_layout()

### Experiment 1: Average and variation in fitting accuracy

In [ ]:
results = []
for seed in range(5):
    for seed_fit in range(5):
        generr, comperr, mcomperr, entropy, runtime, sim, fit = sf.workflow.simulate_fit_and_evaluate(
            s_sim=30,
            n_sim=100,
            g_sim=500,
            n_fit=100,
            g_fit=500,
            sim_kwargs=dict(
                data=dict(
                    alpha_hyper_mean=100.,
                ),
                gamma_hyper=0.01,
                delta_hyper_temp=0.01,
                delta_hyper_p=0.7,
                pi_hyper=0.5,
                rho_hyper=5.,
                mu_hyper_mean=1.,
                mu_hyper_scale=0.5,
                alpha_hyper_scale=0.5,
                epsilon_hyper_alpha=1.5,
                epsilon_hyper_beta=1.5/0.01,
                device='cpu'
            ),
            preclust=True,
            preclust_kwargs=dict(
                thresh=0.1,
                additional_strains_factor=0.1,
                progress=False,
            ),
            fit_kwargs=dict(
                gamma_hyper=0.01,
                pi_hyper=0.5,
                rho_hyper=0.05,
                mu_hyper_mean=5,
                mu_hyper_scale=5.,
                delta_hyper_temp=0.1,
                delta_hyper_p=0.9,
                alpha_hyper_hyper_mean=100.,
                alpha_hyper_hyper_scale=10.,
                alpha_hyper_scale=0.5,
                epsilon_hyper_alpha=1.5,
                epsilon_hyper_beta=1.5 / 0.01,
                device='cpu',
                lagA=10,
                lagB=50,
                opt=pyro.optim.Adamax({"lr": 1e-0}),
                progress=True,
            ),
            postclust=True,
            postclust_kwargs=dict(
                thresh=0.1,
            ),
            seed_sim=seed,
            seed_fit=seed_fit,
            quiet=True,
        )
        results.append((seed_fit, seed, generr, comperr, mcomperr, entropy, runtime))
        print(seed_fit, seed, generr, comperr, mcomperr, entropy, runtime, sep='\t')
         
results1 = pd.DataFrame(results, columns=['seed_fit', 'seed', 'generr', 'comperr', 'mcomperr', 'entropy', 'runtime'])

In [ ]:
fig, axs = plt.subplots(3, 2)

for stat, ax in zip(['generr', 'comperr', 'mcomperr', 'entropy', 'runtime'], axs.flatten()):
    results1.set_index(['seed_fit', 'seed'])[stat].unstack().plot(ax=ax)
    ax.set_title(stat)
    ax.legend_.set_visible(False)
fig.tight_layout()

In [ ]:
plt.scatter(results1['generr'], results1['comperr'], c=results1['seed'])

TODO: Update experiments below to be more like the one above, now that I'm using the DP prior on rho.

### Experiment 2: Benefits of increasing sample data

In [ ]:
results = []
for seed in range(5):
    for n_fit in [20, 50, 100, 150, 200, 500]:
        generr, comperr, mcomperr, entropy, runtime, sim, fit = sf.workflow.simulate_fit_and_evaluate(
            s_sim=30,
            n_sim=1000,
            g_sim=500,
            n_fit=n_fit,
            g_fit=500,
            sim_kwargs=dict(
                data=dict(
                    alpha_hyper_mean=100.,
                ),
                gamma_hyper=0.01,
                delta_hyper_temp=0.01,
                delta_hyper_p=0.7,
                pi_hyper=0.5,
                rho_hyper=5.,
                mu_hyper_mean=1.,
                mu_hyper_scale=0.5,
                alpha_hyper_scale=0.5,
                epsilon_hyper_alpha=1.5,
                epsilon_hyper_beta=1.5/0.01,
                device='cpu'
            ),
            preclust=True,
            preclust_kwargs=dict(
                thresh=0.1,
                additional_strains_factor=0.1,
                progress=False,
            ),
            fit_kwargs=dict(
                gamma_hyper=0.01,
                pi_hyper=0.5,
                rho_hyper=0.05,
                mu_hyper_mean=5,
                mu_hyper_scale=5.,
                delta_hyper_temp=0.1,
                delta_hyper_p=0.9,
                alpha_hyper_hyper_mean=100.,
                alpha_hyper_hyper_scale=10.,
                alpha_hyper_scale=0.5,
                epsilon_hyper_alpha=1.5,
                epsilon_hyper_beta=1.5 / 0.01,
                device='cpu',
                lagA=10,
                lagB=50,
                opt=pyro.optim.Adamax({"lr": 1e-0}),
                progress=True
            ),
            postclust=True,
            postclust_kwargs=dict(
                thresh=0.1,
            ),
            seed_sim=seed,
            seed_fit=seed,
            quiet=True,
        )
        results.append((n_fit, seed, generr, comperr, mcomperr, entropy, runtime))
        print(n_fit, seed, generr, comperr, mcomperr, entropy, runtime, sep='\t')
         
results2 = pd.DataFrame(results, columns=['n_fit', 'seed', 'generr', 'comperr', 'mcomperr', 'entropy', 'runtime'])

In [ ]:
fig, axs = plt.subplots(3, 2)

for stat, ax in zip(['generr', 'comperr', 'mcomperr', 'entropy', 'runtime'], axs.flatten()):
    results2.set_index(['n_fit', 'seed'])[stat].unstack().plot(ax=ax)
    ax.set_title(stat)
    ax.legend_.set_visible(False)
    if stat == 'generr':
        ax.set_yscale('log')
fig.tight_layout()

### Experiment 3: Benefits of increasing depth

In [ ]:
results = []
for seed in range(5):
    for mu_hyper_mean_sim in ([0.1, 0.5, 1.0, 2.0, 5.0, 10.]):
        try:
            generr, comperr, mcomperr, entropy, runtime, sim, fit = sf.workflow.simulate_fit_and_evaluate(
                s_sim=30,
                n_sim=100,
                g_sim=500,
                n_fit=100,
                g_fit=500,
                sim_kwargs=dict(
                    data=dict(
                        alpha_hyper_mean=100.,
                    ),
                    gamma_hyper=0.01,
                    delta_hyper_temp=0.01,
                    delta_hyper_p=0.7,
                    pi_hyper=0.5,
                    rho_hyper=5.,
                    mu_hyper_mean=mu_hyper_mean_sim,
                    mu_hyper_scale=0.5,
                    alpha_hyper_scale=0.5,
                    epsilon_hyper_alpha=1.5,
                    epsilon_hyper_beta=1.5/0.01,
                    device='cpu'
                ),
                preclust=True,
                preclust_kwargs=dict(
                    thresh=0.1,
                    additional_strains_factor=0.1,
                    progress=False,
                ),
                fit_kwargs=dict(
                    gamma_hyper=0.01,
                    pi_hyper=0.5,
                    rho_hyper=0.05,
                    mu_hyper_mean=5,
                    mu_hyper_scale=5.,
                    delta_hyper_temp=0.1,
                    delta_hyper_p=0.9,
                    alpha_hyper_hyper_mean=100.,
                    alpha_hyper_hyper_scale=10.,
                    alpha_hyper_scale=0.5,
                    epsilon_hyper_alpha=1.5,
                    epsilon_hyper_beta=1.5 / 0.01,
                    device='cpu',
                    lagA=10,
                    lagB=50,
                    opt=pyro.optim.Adamax({"lr": 1e-0}),
                    progress=True
                ),
                postclust=True,
                postclust_kwargs=dict(
                    thresh=0.1,
                ),
                seed_sim=seed,
                seed_fit=seed,
                quiet=True,
            )
        except (ValueError, RuntimeError) as err:
            sf.logging_util.info(mu_hyper_mean_sim, seed, f"Estimation failed with: {err}")
        else:
            results.append((mu_hyper_mean_sim, seed, generr, comperr, mcomperr, entropy, runtime))
            print(mu_hyper_mean_sim, seed, generr, comperr, mcomperr, entropy, runtime, sep='\t')

results3 = pd.DataFrame(results, columns=['mu_hyper_mean_sim', 'seed', 'generr', 'comperr', 'mcomperr', 'entropy', 'runtime'])

In [ ]:
fig, axs = plt.subplots(3, 2)

for stat, ax in zip(['generr', 'comperr', 'mcomperr', 'entropy', 'runtime'], axs.flatten()):
    results3.set_index(['mu_hyper_mean_sim', 'seed'])[stat].unstack().plot(ax=ax)
    ax.set_title(stat)
    ax.legend_.set_visible(False)
    if stat == 'generr':
        ax.set_yscale('log')
fig.tight_layout()

### Experiment 4: Benefits of increasing genotype positions

In [ ]:
results = []
for seed in range(5):
    for g_fit in reversed([100, 250, 500, 1000, 2000]):
        try:
            generr, comperr, mcomperr, entropy, runtime, sim, fit = sf.workflow.simulate_fit_and_evaluate(
                s_sim=30,
                n_sim=100,
                g_sim=2000,
                n_fit=100,
                g_fit=g_fit,
                sim_kwargs=dict(
                    data=dict(
                        alpha_hyper_mean=100.,
                    ),
                    gamma_hyper=0.01,
                    delta_hyper_temp=0.01,
                    delta_hyper_p=0.7,
                    pi_hyper=0.5,
                    rho_hyper=5.,
                    mu_hyper_mean=1.,
                    mu_hyper_scale=0.5,
                    alpha_hyper_scale=0.5,
                    epsilon_hyper_alpha=1.5,
                    epsilon_hyper_beta=1.5/0.01,
                    device='cpu'
                ),
                preclust=True,
                preclust_kwargs=dict(
                    thresh=0.1,
                    additional_strains_factor=0.1,
                    progress=False,
                ),
                fit_kwargs=dict(
                    gamma_hyper=0.01,
                    pi_hyper=0.5,
                    rho_hyper=0.05,
                    mu_hyper_mean=5,
                    mu_hyper_scale=5.,
                    delta_hyper_temp=0.1,
                    delta_hyper_p=0.9,
                    alpha_hyper_hyper_mean=100.,
                    alpha_hyper_hyper_scale=10.,
                    alpha_hyper_scale=0.5,
                    epsilon_hyper_alpha=1.5,
                    epsilon_hyper_beta=1.5 / 0.01,
                    device='cpu',
                    lagA=10,
                    lagB=50,
                    opt=pyro.optim.Adamax({"lr": 1e-0}),
                    progress=False
                ),
                postclust=True,
                postclust_kwargs=dict(
                    thresh=0.1,
                ),
                seed_sim=seed,
                seed_fit=seed,
                quiet=True,
            )
        except (ValueError, RuntimeError) as err:
            sf.logging_util.info(g_fit, seed, f"Estimation failed with: {err}")
        else:
            results.append((g_fit, seed, generr, comperr, mcomperr, entropy, runtime))
            print(g_fit, seed, generr, comperr, mcomperr, entropy, runtime, sep='\t')

results4 = pd.DataFrame(results, columns=['g_fit', 'seed', 'generr', 'comperr', 'mcomperr', 'entropy', 'runtime'])

In [ ]:
fig, axs = plt.subplots(3, 2)

for stat, ax in zip(['generr', 'comperr', 'mcomperr', 'entropy', 'runtime'], axs.flatten()):
    results4.set_index(['g_fit', 'seed'])[stat].unstack().plot(ax=ax)
    ax.set_title(stat)
    ax.legend_.set_visible(False)
    if stat == 'generr':
        ax.set_yscale('log')
fig.tight_layout()

### Experiment 5: Strain-number estimation

In [ ]:
results = []
for seed in range(5):
    for s_fit in [5, 10, 15, 30, 50]:
        try:
            generr, comperr, mcomperr, entropy, runtime, sim, fit = sf.workflow.simulate_fit_and_evaluate(
                s_sim=30,
                n_sim=100,
                g_sim=500,
                n_fit=100,
                g_fit=500,
                sim_kwargs=dict(
                    data=dict(
                        alpha_hyper_mean=100.,
                    ),
                    gamma_hyper=0.01,
                    delta_hyper_temp=0.01,
                    delta_hyper_p=0.7,
                    pi_hyper=0.5,
                    rho_hyper=5.,
                    mu_hyper_mean=1.,
                    mu_hyper_scale=0.5,
                    alpha_hyper_scale=0.5,
                    epsilon_hyper_alpha=1.5,
                    epsilon_hyper_beta=1.5/0.01,
                    device='cpu'
                ),
                preclust=False,
#                 preclust_kwargs=dict(
#                     thresh=0.1,
#                     additional_strains_factor=0.1,
#                     progress=False,
#                 ),
                fit_kwargs=dict(
                    s=s_fit,
                    gamma_hyper=0.01,
                    pi_hyper=0.5,
                    rho_hyper=0.05,
                    mu_hyper_mean=5,
                    mu_hyper_scale=5.,
                    delta_hyper_temp=0.1,
                    delta_hyper_p=0.9,
                    alpha_hyper_hyper_mean=100.,
                    alpha_hyper_hyper_scale=10.,
                    alpha_hyper_scale=0.5,
                    epsilon_hyper_alpha=1.5,
                    epsilon_hyper_beta=1.5 / 0.01,
                    device='cpu',
                    lagA=10,
                    lagB=50,
                    opt=pyro.optim.Adamax({"lr": 1e-0}),
                    progress=True,
                ),
                postclust=True,
                postclust_kwargs=dict(
                    thresh=0.1,
                ),
                seed_sim=seed,
                seed_fit=seed,
                quiet=True,
            )
        except (ValueError, RuntimeError) as err:
            sf.logging_util.info(g_fit, seed, f"Estimation failed with: {err}")
        else:
            results.append((s_fit, seed, generr, comperr, mcomperr, entropy, runtime))
            print(s_fit, seed, generr, comperr, mcomperr, entropy, runtime, sep='\t')

results5 = pd.DataFrame(results, columns=['s_fit', 'seed', 'generr', 'comperr', 'mcomperr', 'entropy', 'runtime'])

In [ ]:
fig, axs = plt.subplots(3, 2)

for stat, ax in zip(['generr', 'comperr', 'mcomperr', 'entropy', 'runtime'], axs.flatten()):
    results5.set_index(['s_fit', 'seed'])[stat].unstack().plot(ax=ax)
    ax.set_title(stat)
    ax.legend_.set_visible(False)
    if stat == 'generr':
        ax.set_yscale('log')
fig.tight_layout()

### Experiment 6: Effects of genotype fuzzyness

In [ ]:
results = []
for seed in range(5):
    for gamma_hyper_fit in [1e-8, 1e-5, 1e-2, 5e-2, 1e-1, 5e-1, 1e0]:
        try:
            generr, comperr, mcomperr, entropy, runtime, sim, fit = sf.workflow.simulate_fit_and_evaluate(
                s_sim=30,
                n_sim=100,
                g_sim=500,
                n_fit=100,
                g_fit=500,
                sim_kwargs=dict(
                    data=dict(
                        alpha_hyper_mean=100.,
                    ),
                    gamma_hyper=0.01,
                    delta_hyper_temp=0.01,
                    delta_hyper_p=0.7,
                    pi_hyper=0.5,
                    rho_hyper=5.,
                    mu_hyper_mean=1.,
                    mu_hyper_scale=0.5,
                    alpha_hyper_scale=0.5,
                    epsilon_hyper_alpha=1.5,
                    epsilon_hyper_beta=1.5/0.01,
                    device='cpu'
                ),
                preclust=False,
#                 preclust_kwargs=dict(
#                     thresh=0.1,
#                     additional_strains_factor=0.1,
#                     progress=False,
#                 ),
                fit_kwargs=dict(
                    s=30,
                    gamma_hyper=gamma_hyper_fit,
                    pi_hyper=0.5,
                    rho_hyper=0.05,
                    mu_hyper_mean=5,
                    mu_hyper_scale=5.,
                    delta_hyper_temp=0.1,
                    delta_hyper_p=0.9,
                    alpha_hyper_hyper_mean=100.,
                    alpha_hyper_hyper_scale=10.,
                    alpha_hyper_scale=0.5,
                    epsilon_hyper_alpha=1.5,
                    epsilon_hyper_beta=1.5 / 0.01,
                    device='cpu',
                    lagA=10,
                    lagB=50,
                    opt=pyro.optim.Adamax({"lr": 1e-0}),
                    progress=False,
                ),
                postclust=True,
                postclust_kwargs=dict(
                    thresh=0.1,
                ),
                seed_sim=seed,
                seed_fit=seed,
                quiet=True,
            )
        except (ValueError, RuntimeError) as err:
            sf.logging_util.info(gamma_hyper_fit, seed, f"Estimation failed with: {err}")
        else:
            results.append((gamma_hyper_fit, seed, generr, comperr, mcomperr, entropy, runtime))
            print(gamma_hyper_fit, seed, generr, comperr, mcomperr, entropy, runtime, sep='\t')

results6 = pd.DataFrame(results, columns=['gamma_hyper_fit', 'seed', 'generr', 'comperr', 'mcomperr', 'entropy', 'runtime'])

In [ ]:
fig, axs = plt.subplots(3, 2)

for stat, ax in zip(['generr', 'comperr', 'mcomperr', 'entropy', 'runtime'], axs.flatten()):
    results6.set_index(['gamma_hyper_fit', 'seed'])[stat].unstack().plot(ax=ax)
    ax.set_title(stat)
    ax.legend_.set_visible(False)
    if stat == 'generr':
        ax.set_yscale('log')
fig.tight_layout()

### Experiment 7: Effects of diversity regularization

In [ ]:
results = []
for seed in range(5):
    for rho_hyper_fit in [5, 1, 0.5, 0.1, 0.05, 0.01, 1e-5]:
        try:
            generr, comperr, mcomperr, entropy, runtime, sim, fit = sf.workflow.simulate_fit_and_evaluate(
                s_sim=30,
                n_sim=100,
                g_sim=500,
                n_fit=100,
                g_fit=500,
                sim_kwargs=dict(
                    data=dict(
                        alpha_hyper_mean=100.,
                    ),
                    gamma_hyper=0.01,
                    delta_hyper_temp=0.01,
                    delta_hyper_p=0.7,
                    pi_hyper=0.5,
                    rho_hyper=5.,
                    mu_hyper_mean=1.,
                    mu_hyper_scale=0.5,
                    alpha_hyper_scale=0.5,
                    epsilon_hyper_alpha=1.5,
                    epsilon_hyper_beta=1.5/0.01,
                    device='cpu'
                ),
                preclust=False,
#                 preclust_kwargs=dict(
#                     thresh=0.1,
#                     additional_strains_factor=0.1,
#                     progress=False,
#                 ),
                fit_kwargs=dict(
                    s=35,
                    gamma_hyper=0.05,
                    pi_hyper=0.5,
                    rho_hyper=rho_hyper_fit,
                    mu_hyper_mean=5,
                    mu_hyper_scale=5.,
                    delta_hyper_temp=0.1,
                    delta_hyper_p=0.9,
                    alpha_hyper_hyper_mean=100.,
                    alpha_hyper_hyper_scale=10.,
                    alpha_hyper_scale=0.5,
                    epsilon_hyper_alpha=1.5,
                    epsilon_hyper_beta=1.5 / 0.01,
                    device='cpu',
                    lagA=10,
                    lagB=50,
                    opt=pyro.optim.Adamax({"lr": 1e-0}),
                    progress=False,
                ),
                postclust=True,
                postclust_kwargs=dict(
                    thresh=0.1,
                ),
                seed_sim=seed,
                seed_fit=seed,
                quiet=True,
            )
        except (ValueError, RuntimeError) as err:
            sf.logging_util.info(rho_hyper_fit, seed, f"Estimation failed with: {err}")
        else:
            results.append((rho_hyper_fit, seed, generr, comperr, mcomperr, entropy, runtime))
            print(rho_hyper_fit, seed, generr, comperr, mcomperr, entropy, runtime, sep='\t')

results7 = pd.DataFrame(results, columns=['rho_hyper_fit', 'seed', 'generr', 'comperr', 'mcomperr', 'entropy', 'runtime'])

In [ ]:
fig, axs = plt.subplots(3, 2)

for stat, ax in zip(['generr', 'comperr', 'mcomperr', 'entropy', 'runtime'], axs.flatten()):
    results7.set_index(['rho_hyper_fit', 'seed'])[stat].unstack().plot(ax=ax)
    ax.set_title(stat)
    ax.legend_.set_visible(False)
    if stat == 'generr':
        ax.set_yscale('log')
fig.tight_layout()

### Experiment 8: Effects of heterogeneity regularization

In [ ]:
results = []
for seed in range(5):
    for pi_hyper_fit in [1e-4, 1e-3, 1e-2, 5e-1, 1e0, 1e1, 1e2]:
        try:
            generr, comperr, mcomperr, entropy, runtime, sim, fit = sf.workflow.simulate_fit_and_evaluate(
                s_sim=30,
                n_sim=100,
                g_sim=500,
                n_fit=100,
                g_fit=500,
                sim_kwargs=dict(
                    data=dict(
                        alpha_hyper_mean=100.,
                    ),
                    gamma_hyper=0.01,
                    delta_hyper_temp=0.01,
                    delta_hyper_p=0.7,
                    pi_hyper=0.5,
                    rho_hyper=5.,
                    mu_hyper_mean=1.,
                    mu_hyper_scale=0.5,
                    alpha_hyper_scale=0.5,
                    epsilon_hyper_alpha=1.5,
                    epsilon_hyper_beta=1.5/0.01,
                    device='cpu'
                ),
                preclust=True,
                preclust_kwargs=dict(
                    thresh=0.1,
                    additional_strains_factor=0.1,
                    progress=False,
                ),
                fit_kwargs=dict(
                    gamma_hyper=0.05,
                    pi_hyper=pi_hyper_fit,
                    rho_hyper=0.05,
                    mu_hyper_mean=5,
                    mu_hyper_scale=5.,
                    delta_hyper_temp=0.1,
                    delta_hyper_p=0.9,
                    alpha_hyper_hyper_mean=100.,
                    alpha_hyper_hyper_scale=10.,
                    alpha_hyper_scale=0.5,
                    epsilon_hyper_alpha=1.5,
                    epsilon_hyper_beta=1.5 / 0.01,
                    device='cpu',
                    lagA=10,
                    lagB=50,
                    opt=pyro.optim.Adamax({"lr": 1e-0}),
                    progress=False,
                ),
                postclust=True,
                postclust_kwargs=dict(
                    thresh=0.1,
                ),
                seed_sim=seed,
                seed_fit=seed,
                quiet=True,
            )
        except (ValueError, RuntimeError) as err:
            sf.logging_util.info(pi_hyper_fit, seed, f"Estimation failed with: {err}")
        else:
            results.append((pi_hyper_fit, seed, generr, comperr, mcomperr, entropy, runtime))
            print(pi_hyper_fit, seed, generr, comperr, mcomperr, entropy, runtime, sep='\t')

results8 = pd.DataFrame(results, columns=['pi_hyper_fit', 'seed', 'generr', 'comperr', 'mcomperr', 'entropy', 'runtime'])

In [ ]:
fig, axs = plt.subplots(3, 2)

for stat, ax in zip(['generr', 'comperr', 'mcomperr', 'entropy', 'runtime'], axs.flatten()):
    results8.set_index(['pi_hyper_fit', 'seed'])[stat].unstack().plot(ax=ax)
    ax.set_title(stat)
    ax.legend_.set_visible(False)
    if stat == 'generr':
        ax.set_yscale('log')
    ax.set_xscale('log')
fig.tight_layout()

### Experiment 9: Effects of preclustering threshold

In [ ]:
results = []
for seed in range(5):
    for preclust_thresh in [0.03, 0.05, 0.08, 0.1, 0.12, 0.15, 0.2]:
        try:
            generr, comperr, mcomperr, entropy, runtime, sim, fit = sf.workflow.simulate_fit_and_evaluate(
                s_sim=30,
                n_sim=100,
                g_sim=500,
                n_fit=100,
                g_fit=500,
                sim_kwargs=dict(
                    data=dict(
                        alpha_hyper_mean=100.,
                    ),
                    gamma_hyper=0.01,
                    delta_hyper_temp=0.01,
                    delta_hyper_p=0.7,
                    pi_hyper=0.5,
                    rho_hyper=5.,
                    mu_hyper_mean=1.,
                    mu_hyper_scale=0.5,
                    alpha_hyper_scale=0.5,
                    epsilon_hyper_alpha=1.5,
                    epsilon_hyper_beta=1.5/0.01,
                    device='cpu'
                ),
                preclust=True,
                preclust_kwargs=dict(
                    thresh=preclust_thresh,
                    additional_strains_factor=0.1,
                    progress=False,
                ),
                fit_kwargs=dict(
                    gamma_hyper=0.05,
                    pi_hyper=0.5,
                    rho_hyper=0.05,
                    mu_hyper_mean=5,
                    mu_hyper_scale=5.,
                    delta_hyper_temp=0.1,
                    delta_hyper_p=0.9,
                    alpha_hyper_hyper_mean=100.,
                    alpha_hyper_hyper_scale=10.,
                    alpha_hyper_scale=0.5,
                    epsilon_hyper_alpha=1.5,
                    epsilon_hyper_beta=1.5 / 0.01,
                    device='cpu',
                    lagA=10,
                    lagB=50,
                    opt=pyro.optim.Adamax({"lr": 1e-0}),
                    progress=False,
                ),
                postclust=True,
                postclust_kwargs=dict(
                    thresh=0.1,
                ),
                seed_sim=seed,
                seed_fit=seed,
                quiet=True,
            )
        except (ValueError, RuntimeError) as err:
            sf.logging_util.info(preclust_thresh, seed, f"Estimation failed with: {err}")
        else:
            results.append((preclust_thresh, seed, generr, comperr, mcomperr, entropy, runtime))
            print(preclust_thresh, seed, generr, comperr, mcomperr, entropy, runtime, sep='\t')

results9 = pd.DataFrame(results, columns=['preclust_thresh', 'seed', 'generr', 'comperr', 'mcomperr', 'entropy', 'runtime'])

In [ ]:
fig, axs = plt.subplots(3, 2)

for stat, ax in zip(['generr', 'comperr', 'mcomperr', 'entropy', 'runtime'], axs.flatten()):
    results9.set_index(['preclust_thresh', 'seed'])[stat].unstack().plot(ax=ax)
    ax.set_title(stat)
    ax.legend_.set_visible(False)
    if stat == 'generr':
        ax.set_yscale('log')
    ax.set_xscale('log')
fig.tight_layout()

### Experiment 10: Effects of strain merging (postclustering) threshold

In [ ]:
results = []
for seed in range(5):
    for postclust_thresh in [0.03, 0.05, 0.08, 0.1, 0.12, 0.15, 0.2]:
        try:
            generr, comperr, mcomperr, entropy, runtime, sim, fit = sf.workflow.simulate_fit_and_evaluate(
                s_sim=30,
                n_sim=100,
                g_sim=500,
                n_fit=100,
                g_fit=500,
                sim_kwargs=dict(
                    data=dict(
                        alpha_hyper_mean=100.,
                    ),
                    gamma_hyper=0.01,
                    delta_hyper_temp=0.01,
                    delta_hyper_p=0.7,
                    pi_hyper=0.5,
                    rho_hyper=5.,
                    mu_hyper_mean=5.,
                    mu_hyper_scale=0.5,
                    alpha_hyper_scale=0.5,
                    epsilon_hyper_alpha=1.5,
                    epsilon_hyper_beta=1.5/0.01,
                    device='cpu'
                ),
                preclust=False,
#                 preclust_kwargs=dict(
#                     thresh=0.1,
#                     additional_strains_factor=0.1,
#                     progress=False,
#                 ),
                fit_kwargs=dict(
                    s=50,
                    gamma_hyper=0.05,
                    pi_hyper=0.5,
                    rho_hyper=0.05,
                    mu_hyper_mean=5,
                    mu_hyper_scale=5.,
                    delta_hyper_temp=0.1,
                    delta_hyper_p=0.9,
                    alpha_hyper_hyper_mean=100.,
                    alpha_hyper_hyper_scale=10.,
                    alpha_hyper_scale=0.5,
                    epsilon_hyper_alpha=1.5,
                    epsilon_hyper_beta=1.5 / 0.01,
                    device='cpu',
                    lagA=10,
                    lagB=50,
                    opt=pyro.optim.Adamax({"lr": 1e-0}),
                    progress=False,
                ),
                postclust=True,
                postclust_kwargs=dict(
                    thresh=postclust_thresh,
                ),
                seed_sim=seed,
                seed_fit=seed,
                quiet=True,
            )
        except (ValueError, RuntimeError) as err:
            sf.logging_util.info(postclust_thresh, seed, f"Estimation failed with: {err}")
        else:
            results.append((postclust_thresh, seed, generr, comperr, mcomperr, entropy, runtime))
            print(postclust_thresh, seed, generr, comperr, mcomperr, entropy, runtime, sep='\t')

results10 = pd.DataFrame(results, columns=['postclust_thresh', 'seed', 'generr', 'comperr', 'mcomperr', 'entropy', 'runtime'])

In [ ]:
fig, axs = plt.subplots(3, 2)

for stat, ax in zip(['generr', 'comperr', 'mcomperr', 'entropy', 'runtime'], axs.flatten()):
    results10.set_index(['postclust_thresh', 'seed'])[stat].unstack().plot(ax=ax)
    ax.set_title(stat)
    ax.legend_.set_visible(False)
    if stat == 'generr':
        ax.set_yscale('log')
    ax.set_xscale('log')
fig.tight_layout()

### Experiment 11: Learning rate

In [ ]:
results = []
for seed in range(5):
    for learning_rate in [0.05, 0.1, 0.5, 1., 1.5, 2.]:
        try:
            generr, comperr, mcomperr, entropy, runtime, sim, fit = sf.workflow.simulate_fit_and_evaluate(
                s_sim=30,
                n_sim=100,
                g_sim=500,
                n_fit=100,
                g_fit=500,
                sim_kwargs=dict(
                    data=dict(
                        alpha_hyper_mean=100.,
                    ),
                    gamma_hyper=0.01,
                    delta_hyper_temp=0.01,
                    delta_hyper_p=0.7,
                    pi_hyper=0.5,
                    rho_hyper=5.,
                    mu_hyper_mean=1.,
                    mu_hyper_scale=0.5,
                    alpha_hyper_scale=0.5,
                    epsilon_hyper_alpha=1.5,
                    epsilon_hyper_beta=1.5/0.01,
                    device='cpu'
                ),
                preclust=True,
                preclust_kwargs=dict(
                    thresh=0.1,
                    additional_strains_factor=0.1,
                    progress=False,
                ),
                fit_kwargs=dict(
                    gamma_hyper=0.05,
                    pi_hyper=0.5,
                    rho_hyper=0.05,
                    mu_hyper_mean=5,
                    mu_hyper_scale=5.,
                    delta_hyper_temp=0.1,
                    delta_hyper_p=0.9,
                    alpha_hyper_hyper_mean=100.,
                    alpha_hyper_hyper_scale=10.,
                    alpha_hyper_scale=0.5,
                    epsilon_hyper_alpha=1.5,
                    epsilon_hyper_beta=1.5 / 0.01,
                    device='cpu',
                    lagA=10,
                    lagB=50,
                    opt=pyro.optim.Adamax({"lr": learning_rate}),
                    progress=True,
                ),
                postclust=True,
                postclust_kwargs=dict(
                    thresh=0.1,
                ),
                seed_sim=seed,
                seed_fit=seed,
                quiet=True,
            )
        except (ValueError, RuntimeError) as err:
            sf.logging_util.info(learning_rate, seed, f"Estimation failed with: {err}")
        else:
            results.append((learning_rate, seed, generr, comperr, mcomperr, entropy, runtime))
            print(learning_rate, seed, generr, comperr, mcomperr, entropy, runtime, sep='\t')

results11 = pd.DataFrame(results, columns=['learning_rate', 'seed', 'generr', 'comperr', 'mcomperr', 'entropy', 'runtime'])

In [ ]:
fig, axs = plt.subplots(3, 2)

for stat, ax in zip(['generr', 'comperr', 'mcomperr', 'entropy', 'runtime'], axs.flatten()):
    results11.set_index(['learning_rate', 'seed'])[stat].unstack().plot(ax=ax)
    ax.set_title(stat)
    ax.legend_.set_visible(False)
    if stat == 'generr':
        ax.set_yscale('log')
    ax.set_xscale('log')
fig.tight_layout()

### Visualize all

In [ ]:
# TODO: Big matrix plot.

all_results = [
    (results1, 'seed_fit', 'linear', 1),
    (results2, 'n_fit', 'log', 2),
    (results3, 'mu_hyper_mean_sim', 'log', 3),
    (results4, 'g_fit', 'log', 4),
    (results5, 's_fit', 'log', 5),
    (results6, 'gamma_hyper_fit', 'log', 6),
    (results7, 'rho_hyper_fit', 'log', 7),
    (results8, 'pi_hyper_fit', 'log', 8),
    (results9, 'preclust_thresh', 'linear', 9),
    (results10, 'postclust_thresh', 'linear', 10),
]

all_stats = [
    ('generr', 'log'),
    ('comperr', 'log'),
    ('mcomperr', 'log'),
    ('entropy', 'linear'),
    ('runtime', 'log')
]

nres = len(all_results)
nstat = len(all_stats)

fig, axs = plt.subplots(nstat, nres, figsize=(3 * nres, 2 * nstat), sharex='col', sharey='row')

for (stat, scale_y), row in zip(all_stats, axs):
    for (results, indexer, scale_x, title), ax in zip(all_results, row):
        results.set_index([indexer, 'seed'])[stat].unstack().plot(ax=ax)
        ax.set_ylabel(stat)
        ax.set_xlabel(indexer)
        ax.legend_.set_visible(False)
        ax.set_xscale(scale_x)
        ax.set_yscale(scale_y)
#        ax.set_title(title)

fig.tight_layout()
#ax.legend(bbox_to_anchor=(1, 1), title='replicate')

## Demonstrations

### Demo: Good accuracy with realistic conditions

In [ ]:
results = []
for seed in range(20):
    generr, comperr, scounter, entropy, runtime, sim, fit = sf.workflow.simulate_fit_and_evaluate(
        s_sim=20,
        n_sim=200,
        g_sim=1000,
        n_fit=200,
        g_fit=1000,
        sim_kwargs=dict(
            data=dict(
                alpha_hyper_mean=100.
            ),
            gamma_hyper=0.01,
            delta_hyper_temp=0.01,
            delta_hyper_p=0.9,
            pi_hyper=0.5,
            rho_hyper=2.,
            mu_hyper_mean=10.,
            mu_hyper_scale=0.5,
            m_hyper_r=10.,
            alpha_hyper_scale=0.5,
            epsilon_hyper_alpha=1.5,
            epsilon_hyper_beta=1.5/0.01,
            device='cpu'
        ),
        preclust_kwargs=dict(
            thresh=0.1,
            additional_strains_factor=0.1,
            progress=False,
        ),
        fit_kwargs=dict(
            gamma_hyper=0.01,
            pi_hyper=1.0,
            rho_hyper=0.5,
            mu_hyper_mean=5,
            mu_hyper_scale=5.,
            m_hyper_r=10.,
            delta_hyper_temp=0.1,
            delta_hyper_p=0.9,
            alpha_hyper_hyper_mean=100.,
            alpha_hyper_hyper_scale=10.,
            alpha_hyper_scale=0.5,
            epsilon_hyper_alpha=1.5,
            epsilon_hyper_beta=1.5 / 0.01,
            device='cuda',
            lag=10,
            lr=2e-0,
            progress=True
        ),
        postclust_kwargs=dict(
            thresh=0.1,
        ),
        seed_sim=seed,
        seed_fit=seed,
        quiet=True,
    )
    results.append((seed, generr, comperr, scounter, entropy, runtime))
    print(seed, generr, comperr, scounter, entropy, runtime, sep='\t')
         
results_d0 = pd.DataFrame(results, columns=['seed', 'generr', 'comperr', 'scounterr', 'entropy', 'runtime'])

In [ ]:
fig, axs = plt.subplots(3, 2)

for stat, ax in zip(['generr', 'comperr', 'scounterr', 'entropy', 'runtime'], axs.flatten()):
    ax.hist(results_d0[stat], bins=5)
    ax.set_title(stat)
fig.tight_layout()